<a href="https://colab.research.google.com/github/cbolton56/naive_bayes/blob/master/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [6]:
from math import log
#GLOBAL VARIABLES to easily change certain values
N_COLUMNS = 28
FOREGROUND = 1
BACKGROUND = 0
#Laplace Smoothing Factor
LAPLACE_K = 2

def main():  
    """The main function opens files and calls training and testing functions.

    Args: none

    Returns: void
    """
    #Performing all loading operations first
    test_images = open_image_file("./testimages.txt")
    test_labels = open_label_file("./testlabels.txt")
    raw_images = get_raw_images("./testimages.txt")
    train_labels = open_label_file("./traininglabels.txt")
    train_images = open_image_file("./trainingimages.txt")

    #Calculate the prior liklihoods
    priors = calculate_prior(train_labels)
    #Crate a dictionary that stores the sorted images 
    dictionary_training = sort_images(train_images, train_labels)
    #Calculate the likelihood values for each element in the 28x28 matrix
    likelihoods = calculate_likelihood(dictionary_training)
    #Estimates holds the predicted value of each image file
    estimates = classify_test_data(priors, likelihoods, test_images)
    #Print results to screen
    display_results(estimates, raw_images)
    print_confusion_matrix(estimates, test_labels, priors)

def open_label_file(filename):
    """Opens a file that holds labels for training and testing data, storing each integer in a list.

    Args: @filename: a string that holds the name of the file to be opened

    Returns: @labels, a list of integers which are the true values of the images
    """
    #An empty array to hold labels
    labels = []
    #Open filestream
    infile = open(filename)
    #Convert each line to an integer and store in an array
    for line in infile.readlines(): 
        labels.append(int(line))
    #Close the filestream.
    infile.close()
    return labels

def open_image_file(filename):
    """Opens a file that holds image data. This function reads in 28 x 28 images and calls to_list to
    convert every pixel to a binary (0 or 1) integer which represent a foreground or background.

    Args: @filename: a string that holds the name of the file to be opened

    Returns: @images, a list of 28 x 28 matricies 
    """
    #Store all images in the empty list, images
    images = []
    #Temp will temporarily store the image data while iterating through rows
    temp = []
    #establish a filestream 
    infile = open(filename)
    counter = 0
    for line in infile.readlines():
        #Converts the string to a list object, removing the endline characters and changing  all chars to 0 or 1
        int_array = to_list(line[:N_COLUMNS])
        temp.append(int_array)
        counter += 1
        #Every 28 characters append the image to the image list and clear the temporary array.
        if counter == N_COLUMNS: 
            counter = 0
            images.append(temp[:])
            temp.clear()
    #Close the filestream.
    infile.close()
    return images

def to_list(line):
    """A function to convert a string of input pixels (which are either "#", "+" or " ") to binary variables stored in a list

    Args: @line: a string of characters from the image file (either "+", "#" or " ")

    Returns: @int_array, a list of 0 or 1 representing foreground or background pixels
    """
    int_array = []
    for char in line:
        #A space is a background (white) pixel and is converted to 0
        if char == " ": 
            int_array.append(BACKGROUND)
        else:
        #Anything besides a space is a non white pixel represented by 1
            int_array.append(FOREGROUND)
    return int_array

def sort_images(train_images, train_labels):
    """This function sorts the training images based on what digit they belong to (0 - 9). 

    Args: @train_images: a list of 2D matricies with image information
          @train_labels: a list of labels that are in the same order as the images.

    Returns: @dictionary: a dictionary object where each image is sorted by key.
    """
    dictionary = {0: [], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]}
    for idx, element in enumerate(train_labels):
        dictionary[element].append(train_images[idx])
    return dictionary

def calculate_prior(train_labels):
    """This function calculates the prior probability (P(A)) by counting how many images of each type exist and dividing by 
    the total amount of images in the label file. 

    Args: @train_labels: a list of labels (integers).

    Returns: @priors: a list of prior probabilities where index of the element is the digit and the value at the index is the probability of that
    digit occurring.
    """
    priors = [0 for x in range(10)]
    for value in train_labels: 
        priors[value] += 1
    #To find the probability of some digit occurring, divide the count of that digit by the total occurrences of each digit, 0-9.
    denominator = len(train_labels)
    priors = list(map(lambda x: x / denominator, priors))
    return priors

def calculate_likelihood(dictionary):
    """This function calculates likelihoods for each element in the 28x28 matricies. It calls a helper function sum_matrix to 
    combine all the matricies per digit into two: one storing the counts of how many foreground pixels and one storing the counts
    of how many background pixels for all the matricies of each digit. It also calls laplace smooth, to add a laplace smoothing effect to
    the probabilities.

    Args: @dictionary: a dict of images that correspond to each digit 0-9

    Returns: @dictionary after converting all the images into two matricies. It also stores the denominator in a third slot for each key.
    """
    for key in dictionary:
        #Sum each matrix and get the counts of all foreground and all background pixels into 2 matricies. 
        matrix_1, matrix_0, denominator = sum_matrix(dictionary[key])
        dictionary[key] = (matrix_1, matrix_0, denominator)
    laplace_smooth(dictionary, 2)
    return dictionary

def sum_matrix(key):
    """This function is a helper function for calculate_likelihood. It creates a matrix to hold the counts of the foreground pixels (1)
    and a matrix to hold the counts of the background pixels (0) and finds the denominator for each probability, which is the length of each key,
    or how many images are originally in the dictionary object for each key.

    Args: @dictionary: a dict of images that correspond to each digit 0-9

    Returns: @dictionary: which now holds a tuple, the 1 matrix, the 0 matrix, and the denominator for the values in that matrix, which will be used later.
    """
    #matrix_1 one holds the counts for how many 1's there are
    matrix_1 = [[0 for x in range(len(key[0]))] for y in range(len(key[0][0]))]
    #matrix_0 holds counts for how many 0's there are
    matrix_0 = [[0 for x in range(len(key[0]))] for y in range(len(key[0][0]))]
    denominator = len(key)
    for element in key:
        for i in range(len(key[0])): 
            for j in range(len(key[0][0])):
                if element[i][j] == 0: 
                    matrix_0[i][j] += 1
                else:
                    matrix_1[i][j] += element[i][j]
    return (matrix_1, matrix_0, denominator) 

def laplace_smooth(dictionary, LAPLACE_K): 
    """This function adds laplace smoothing to the likelihood probability matrix. 

    Args: @dictionary: a dict of tuples that correspond to each digit 0-9 and the denominator for that value.
            @LAPLACE_K: The laplace smoothing factor

    Returns: @dictionary: a dictionary that holds 2 matrices of probabilities for each 28x28 values for each digit 0-9
    """
    for key in dictionary:
        temp = dictionary[key]
        matrix_1 = temp[0]
        matrix_0 = temp[1]
        denominator = temp[2]
        for row in range(len(matrix_1)): 
            for column in range(len(matrix_1[0])):
                matrix_1[row][column] += LAPLACE_K
                matrix_0[row][column] += LAPLACE_K
        denominator += (2 * LAPLACE_K)
    #After adding 2k to the numerator and denominator divide each matrix by the denominator to get the final likelihood probabilities.
        for row in range(len(matrix_1)): 
            for column in range(len(matrix_1[0])): 
                matrix_1[row][column] = matrix_1[row][column] / denominator
                matrix_0[row][column] = matrix_0[row][column] / denominator
        dictionary[key] = (matrix_1, matrix_0, denominator)
    return dictionary

    
def classify_test_data(priors, likelihoods, test_images):
    """This function transitions from training to testing, using the prior and likelihoood to classify the test images. It 
    calls the helper function return_MAP_estimate, which calculates the MAP for each image in the list of test images.

    Args: @priors: a list of prior probabilities
            @likelihoods: probability of likelihoods for each  element. The dictionary object has 2 matrix: the zero matrix and 
            one matrix, per key.
            @test_images: a list of test images.

    Returns: @estimates, a list of integer values that represents the prediction for each image.
    """
    #Estimates holds the estimate for each image, its index number will correspond to the label.
    estimates = []
    for image in test_images:
        estimate = return_MAP_estimate(image, priors, likelihoods)
        estimates.append(estimate)
    return estimates

def return_MAP_estimate(image, priors, likelihoods):
    """This function transitions from training to testing, using the prior and likelihoood to classify the test images. It 
    calls the helper function return_MAP_estimate, which calculates the MAP for each image in the list of test images.

    Args: @priors: a list of prior probabilities
    @image: a single image file (28 x 28 matrix)
    @likelihoods: a dictionary holding the 0 and 1 matrix.

    Returns: @image_class: the guess for each iamge.
    """
    estimates = [0 for x in range(len(priors))]
    for i in range(len(priors)):
        ones_matrix = likelihoods[i][0]
        zero_matrix = likelihoods[i][1]
        probability = priors[i]
        for row in range(len(image)):
            for column in range(len(image[0])):
                if image[row][column] == 1:
                    probability += log(ones_matrix[row][column]) 
                else: 
                    probability += log(zero_matrix[row][column])
        estimates[i] = probability
    #Iterate through estimates and return the INDEX value of the largest value.
    image_class = estimates.index(max(estimates))
    return image_class

def print_confusion_matrix(estimates, test_labels, priors):
    """This function prints a confusion matrix, showing which images were classified correctly and which were classified incorrectly.

    Args: @estimates: the estimates for each image. 
        @test_labels: the labels for the data we tested on for checking correctness
        @priors: all prior probabilities

    Returns: void
    """
    confusion_matrix = [[0 for x in range(len(priors))] for x in range(len(priors))]
    for idx, value in enumerate(estimates): 
        row = value
        column = test_labels[idx]
        confusion_matrix[row][column] += 1
    
    for row in confusion_matrix:
        print(row)
    
    #Compute the overall accuracy of the matrix.
    matches = 0 
    for i in range(len(confusion_matrix)): 
        matches += confusion_matrix[i][i]
    
    total = len(test_labels) 
    accuracy = matches / total 
    print("Total Accuracy: ", accuracy * 100, "%")

def get_raw_images(filename):
    """This function stores the raw image data into an object that can then be printed and displayed to the screen

    Args: @filename: the name of the file to establish a stream

    Returns: images, a list that has not been converted to 0 or 1
    """
    #Store all images in the empty list, images
    images = []
    #Temp will temporarily store the image data while iterating through rows
    temp = []
    #establish a filestream 
    infile = open(filename)
    counter = 0
    for line in infile.readlines():
        #Converts the string to a list object, removing the endline characters and changing  all chars to 0 or 1
        arr = list(line[:28])
        temp.append(arr)
        counter += 1
        #Every 28 characters append the image to the image list and clear the temporary array.
        if counter == 28: 
            counter = 0
            images.append(temp[:])
            temp.clear()
    #Close the filestream.
    infile.close()
    return images

def display_results(estimates, raw_images):
    """This function prints a confusion matrix, showing which images were classified correctly and which were classified incorrectly.

    Args: @estimates: the estimated values of the images
    @raw_images: the images as they appeared in the input files. 

    Returns: void
    """
    counter = 0
    for idx, image in enumerate(raw_images):
        for row in image: 
            print(row)
        print("\nPredicted Value: ", estimates[idx])
        print("Serial Number: ", idx, "\n")
        counter += 1
        if counter == 100: 
          break 

if __name__ == '__main__': 
    main()

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' 